<a href="https://colab.research.google.com/github/ishancoderr/Data_preparetion_for_research/blob/master/GAN_RESEARCH_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [249]:
# Load the data from GitHub
data = pd.read_csv("https://raw.githubusercontent.com/ishancoderr/Data_preparetion_for_research/master/rainfall_dataset.csv")

In [250]:
data

,Unnamed: 0,new_cum_rainfall,new_hours,new_intensity,status
0,0,185.333,4.393147,42.186847,Generated_No
1,1,170.750,4.036087,42.305825,Generated_No
2,2,192.333,4.625477,41.581228,Generated_No
3,3,171.500,4.499677,38.113846,Generated_No
4,4,204.000,4.755184,42.900545,Generated_No
...,...,...,...,...,...
151,151,182.000,5.000000,36.400000,yes
152,152,211.000,5.000000,42.200000,yes
153,153,320.000,6.000000,53.333333,yes
154,154,307.000,7.000000,43.857143,yes


In [251]:
data['status'] = data['status'].replace({'Generated_No': 0, 'yes': 1})

In [252]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[['new_cum_rainfall', 'new_hours', 'new_intensity','status']] = scaler.fit_transform(data[['new_cum_rainfall', 'new_hours', 'new_intensity','status']])

In [253]:
data

,Unnamed: 0,new_cum_rainfall,new_hours,new_intensity,status
0,0,0.302890,0.278629,0.601271,0.0
1,1,0.236301,0.207217,0.605373,0.0
2,2,0.334854,0.325095,0.580387,0.0
3,3,0.239726,0.299935,0.460822,0.0
4,4,0.388128,0.351037,0.625881,0.0
...,...,...,...,...,...
151,151,0.287671,0.400000,0.401724,1.0
152,152,0.420091,0.400000,0.601724,1.0
153,153,0.917808,0.600000,0.985632,1.0
154,154,0.858447,0.800000,0.658867,1.0


In [254]:
# Split the data into input and target variables
X = data[['new_cum_rainfall', 'new_hours', 'new_intensity']].values

In [255]:
y=data['status']

In [256]:
# Define the generator network
def generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(None,3)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [257]:
# Define the discriminator network
def discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(None,1)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [258]:
# Compile the generator and discriminator
generator = generator()
discriminator = discriminator()

generator.compile(loss='mean_squared_error', optimizer='adam')
discriminator.compile(loss='mean_squared_error', optimizer='adam')

In [259]:
# Combine the generator and discriminator into a GAN
gan = tf.keras.Sequential([generator, discriminator])
gan.compile(loss='mean_squared_error', optimizer='adam')

In [260]:
# Train the GAN
for epoch in range(700):
    # Train the discriminator
    X_fake = generator.predict(X)
     

    y_fake = np.zeros(len(y))
    X_real = X
    y_real = y

    discriminator.trainable = True
    discriminator.train_on_batch(X_real, y_real)
    discriminator.train_on_batch(X_fake, y_fake)

    # Train the generator
    discriminator.trainable = False
    gan.train_on_batch(X, y_real)

5/5 [==============================] - 0s 3ms/step


5/5 [==============================] - 0s 3ms/step


In [264]:
print(discriminator.evaluate(X_real,y_real,verbose=0)) 
print(discriminator.evaluate(X_fake,y_fake, verbose=0)) 

0.1482732594013214
0.021810507401823997


In [263]:
# Define the input for the new data
user_input = np.array([[320.000,	6.000000,	53.33]]) # example user input

user_input_scaled = scaler.fit_transform(user_input)
# Generate a prediction for the new data
prediction = generator.predict(user_input_scaled)
print(prediction)
# Convert the prediction to a binary output (0 or 1)
if prediction < 0.5:
    output = 0 # Generated_No
else:
    output = 1 # yes

# Print the output
print(output)

1/1 [==============================] - 0s 24ms/step
[[2.7887467e-16]]
0
